In [1]:
# !pip install fuzzywuzzy # Требуется для Google Colab

import chardet
import pandas as pd 
from fuzzywuzzy import fuzz  # Библиотека для нечёткого сравнения строк

# Функция для определения кодировки файла
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    return result['encoding']

# Функция для нормализации названий регионов
def normalize_region_name(name):
    if not name or not isinstance(name, str):
        return None
    name = name.lower() \
        .replace('городской округ', 'округ') \
        .replace('муниципальный район', 'район') \
        .replace('сельское население', '') \
        .replace('городское население', '') \
        .replace('сельское поселение', 'поселение') \
        .replace('рп', 'рабочий поселок') \
        .replace('с подчиненными его администрации населенными пунктами', '') \
        .replace('-', '').replace('ё', 'е').strip()
    
    words_to_remove = ['пгт', 'г.', 'г']
    name_parts = [part for part in name.split() if part not in words_to_remove]
    return ' '.join(e for e in name_parts if e.isalnum() or e in ['округ', 'район'])


e:\Python\foliumVRN\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:

# Функция для поиска наиболее подходящего совпадения региона
def find_matching_region(target, candidates):
    if target in candidates:
        return target
    for candidate in candidates:
        if target in candidate or candidate in target:
            return candidate
    max_similarity = 0
    best_match = None
    for candidate in candidates:
        similarity = fuzz.token_set_ratio(target, candidate)
        if similarity > 85 and similarity > max_similarity:
            max_similarity = similarity
            best_match = candidate
    return best_match

# Функция для фильтрации данных по указанной области
def filter_data_by_region(data, target_region):
    filtered_data = []
    collecting = False
    for _, row in data.iterrows():
        region = row['region'].lower()
        if region == target_region.lower():
            collecting = True
            continue
        if collecting:
            if region.endswith('область') or region.endswith('край'):
                break
            else:
                normalized_region = normalize_region_name(region)
                if normalized_region:
                    row_dict = row.to_dict()
                    row_dict['region'] = normalized_region  # Сохраняем нормализованный регион
                    filtered_data.append(row_dict)
    
    if not filtered_data:
        return pd.DataFrame(columns=data.columns).set_index('region')
    return pd.DataFrame(filtered_data).set_index('region')

# Загрузка путей к CSV-файлам и соответствующих годов от пользователя
csv_files = []
years = []
while True:
    file_path = input("Введите путь к CSV файлу (или оставьте пустым для завершения): ").strip()
    if not file_path:
        break
    year = input(f"Введите год для файла {file_path} (например, 2023): ").strip()
    if not year:
        year = 'unknown'
    csv_files.append(file_path)
    years.append(year)

if not csv_files:
    print("Нет загруженных файлов. Программа завершена.")
    exit()


In [3]:

# Чтение данных из CSV-файлов
data_frames = []
russian_headers = []
for file in csv_files:
    try:
        encoding = detect_encoding(file)
        df = pd.read_csv(file, sep=';', encoding=encoding, header=0)
        df.columns = [col.strip().lower() for col in df.columns]
        with open(file, 'r', encoding=encoding) as f:
            next(f)  # Пропускаем первую строку с английскими заголовками
            russian_header = next(f).strip().split(';')
            russian_headers.append([col.strip() for col in russian_header])
        df['region'] = df['region'].str.strip().str.lower()
        df = df.dropna(subset=['region'])  # Удаляем строки с пустыми регионами
        data_frames.append(df)
    except Exception as e:
        print(f"Ошибка при чтении файла {file}: {e}")

if not data_frames:
    print("Не удалось загрузить данные из файлов. Программа завершена.")
    exit()

# Запрос целевой области для фильтрации
target_region = input("Введите название области для фильтрации (например, 'воронежская область'): ").strip().lower()
if not target_region:
    target_region = 'воронежская область'

# Фильтрация данных по целевой области
filtered_data = []
for df in data_frames:
    filtered_df = filter_data_by_region(df, target_region)
    if not filtered_df.empty:
        filtered_data.append(filtered_df)

if not filtered_data:
    print("Данные для указанной области не найдены. Программа завершена.")
    exit()


In [4]:

# Функция для выбора пользователем столбцов из DataFrame
def select_columns(df, file_name):
    print(f"\nДоступные столбцы в файле '{file_name}': {df.columns.tolist()}")
    selected_columns = input("Введите через запятую названия столбцов, которые хотите добавить: ").strip()
    if not selected_columns:
        return []
    return [col.strip().lower() for col in selected_columns.split(",")]

# Инициализация списков для комбинированных данных и выбранных столбцов
combined_data = []
selected_columns_per_file = []
for j, df in enumerate(filtered_data):
    selected_columns = select_columns(df, csv_files[j])
    selected_columns_per_file.append(selected_columns)

# Сбор всех уникальных регионов из отфильтрованных данных
all_regions = set()
for df in filtered_data:
    all_regions.update(df.index)

# Формирование списка всех уникальных столбцов, включая 'region' и 'year'
all_unique_columns = ['region', 'year']
for selected_columns in selected_columns_per_file:
    for col in selected_columns:
        if col not in all_unique_columns:
            all_unique_columns.append(col)

# Создание словаря для русских названий столбцов
russian_column_names = {}
for j, selected_columns in enumerate(selected_columns_per_file):
    for col in selected_columns:
        english_headers = [col.strip().lower() for col in data_frames[j].columns]
        russian_index = english_headers.index(col)
        russian_name = russian_headers[j][russian_index]
        russian_column_names[col] = russian_name

# Функция для преобразования строковых значений в числа
def clean_numeric_value(value):
    if isinstance(value, str):
        value = value.replace(' ', '').replace(',', '.')
        try:
            if '.' in value:
                return float(value)
            else:
                return int(value)
        except ValueError:
            pass
    return value



Доступные столбцы в файле 'csv\Popularity2020.csv': ['total_population', 'male_population', 'female_population', 'male_percent', 'female_percent']

Доступные столбцы в файле 'csv\Popularity2010.csv': ['total_population', 'male_population', 'female_population', 'male_percent', 'female_percent']

Доступные столбцы в файле 'csv\Popularity2002.csv': ['total_population', 'male_population', 'female_population', 'male_percent', 'female_percent']


In [5]:

# Объединение данных из всех файлов с учётом регионов и годов
for region in all_regions:
    for j, df in enumerate(filtered_data):
        match = find_matching_region(region, df.index.tolist())
        if match and match in df.index:
            match_index = df.index.tolist().index(match)
            combined_row = {'region': region, 'year': years[j]}
            for col in selected_columns_per_file[j]:
                if col in df.columns:
                    value = df.iloc[match_index][col]
                    combined_row[col] = clean_numeric_value(value)
            combined_data.append(combined_row)

# Создание итогового DataFrame из объединённых данных
combined_data_df = pd.DataFrame(combined_data)

# Добавление строки с русскими названиями столбцов
russian_names_row = {col: russian_column_names.get(col, col) for col in all_unique_columns}
russian_names_row['region'] = russian_headers[0][0]
russian_names_row['year'] = 'Год'

# Объединение строки с русскими названиями и данных
combined_data_df = pd.concat([pd.DataFrame([russian_names_row]), combined_data_df])
combined_data_df = combined_data_df[all_unique_columns]

# Сохранение итоговой таблицы в CSV-файл
output_file = input("Введите путь для сохранения комбинированной таблицы (например, 'combined_table.csv'): ").strip()
if not output_file:
    output_file = 'combined_table.csv'
try:
    combined_data_df.to_csv(output_file, sep=';', encoding='utf-8-sig', index=False)
    print(f"Комбинированная таблица успешно сохранена в файл: {output_file}")
except Exception as e:
    print(f"Ошибка при сохранении файла: {e}")

Комбинированная таблица успешно сохранена в файл: combined_table.csv
